## Azure AI Search ഉപയോഗിച്ച് Phi-4 RAG

ഈ നോട്ട്‌ബുക്ക് Azure AI Search ഉപയോഗിച്ച് Retrieval Augmented Generation (RAG) සඳහා Phi-4-miniയും Phi-4-multimodalഉം എങ്ങനെ ഉപയോഗിക്കാമെന്ന് കാണിക്കുന്നു. ഇത് സിംഗിൾ-മൊദാലിറ്റി (ടെക്സ്റ്റ് മാത്രം)യും മൾട്ടി-മൊദാലിറ്റി (ടെക്സ്റ്റും ചിത്രം)യും ഉള്ള senarios-ഉം ഉൾക്കൊള്ളുന്നു.

**മുൻകൂർ ആവശ്യങ്ങൾ:**
*   Azure AI Search വെക്ടർ ഇൻഡക്സ് (ഒരു ഇൻഡക്സ് സൃഷ്ടിക്കാൻ [these instructions](https://learn.microsoft.com/azure/search/search-get-started-portal-import-vectors?tabs=sample-data-storage%2Cmodel-aoai%2Cconnect-data-storage) പിന്തുടരുക)
*   Phi-4-mini അല്ലെങ്കിൽ Phi-4-multimodal എന്റ്പോയിന്റുകൾ Azure AI Foundry-ൽ വിന്യസിച്ചിരിക്കുന്നത്


In [ ]:
! pip install azure-ai-inference azure-search-documents

### ടെക്സ്റ്റ് മാത്രം RAG Phi-4-mini ഉപയോഗിച്ച്

ഈ വിഭാഗം Phi-4-mini-നെ RAG-ക്കായുള്ള ചാറ്റ് പൂർത്തീകരണ മോഡലായി ഇൻപുട്ടായി ടെക്സ്റ്റ് മാത്രം ഉപയോഗിച്ച് എങ്ങനെ ഉപയോഗിക്കാമെന്ന് കാണിക്കുന്നു. ഇത് Azure AI Foundry Inference and Azure AI Search ഇവയുമായി കണക്റ്റ് ചെയ്യുകയും, അനുയോജ്യ രേഖകൾ തിരികെ വീണ്ടെടുക്കുകയും, തിരികെ കിട്ടിയ കോൺടെക്സ്റ്റ് ഉപയോഗിച്ചു ഒരു ഉത്തരം സൃഷ്ടിക്കുകയും ചെയ്യുന്നത് ഉൾക്കൊള്ളുന്നു.


In [ ]:
# Step 1: Connect to Azure AI Foundry Inference & Azure AI Search
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], # Phi-4-mini endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 10):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query,vector_queries=[vector_query],select=["content"],top=top)
    return [doc["content"] for doc in results]

# Step 3: Generate answer using RAG!
def generate_rag_response(query: str):
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    response = chat_client.complete(messages=[UserMessage(content=prompt)])
    return response.choices[0].message.content

# Example usage
user_query = "What is the capital of France?"
answer = generate_rag_response(user_query)
print(f"Q: {user_query}\nA: {answer}")


### Phi-4-multimodal ഉപയോഗിച്ചുള്ള ബഹുമോഡൽ RAG

ഈ ഭാഗം Phi-4-multimodal-നെ RAG-യ്ക്കായുള്ള ഒരു ചാറ്റ് പൂർത്തീകരണ മോഡലായി ഉപയോഗിച്ച് ടെക്സ്റ്റ്‌യും ചിത്രം ഇൻപുട്ടും ഉൾപ്പെടുത്തുന്നത് എങ്ങനെ നടത്താമെന്ന് കാണിക്കുന്നു.  ഇത് Azure AI Inference-ലേക്കും Azure AI Search-ലേക്കും കണക്റ്റ് ചെയ്യലും, പ്രസക്തമായ രേഖകൾ തിരിച്ചു കണ്ടെത്തലും, ബഹുമോഡൽ പ്രതികരണം സൃഷ്ടിക്കലും ഉൾക്കൊള്ളുന്നു.

**കുറിപ്പ്:** നിങ്ങളുടെ Azure AI Search ഇൻഡക്സിൽ `text_vector`യും `image_vector`യും ഫീൽഡുകൾ ഉണ്ടെങ്കിൽ, നിങ്ങൾ മൾട്ടി-വെക്ടർ ക്വറി നടത്താവുന്നതാണ്.


In [ ]:
# Step 1: Connect to Azure AI Inference & Azure AI Search
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery


chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], #Phi-4-multimodal serverless endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 3):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query, vector_queries=[vector_query], select=["content"], top=top)    
    return [doc["content"] for doc in results]

## Example for Muli-modal search if you have a text_vector AND image_vector field in your vector_index
## NOTE, image vectorization is in preview at the time of writing this code, please use azure-search-documents pypi version >11.6.0b6 
# def retrieve_documents_multimodal(query: str, image_url: str, top: int = 3):
#     text_vector_query = VectorizableTextQuery(
#         text=query,
#         k_nearest_neighbors=top,
#         fields="text_vector",
#         weight=0.5  # Adjust weight as needed
#     )
#     image_vector_query = VectorizableImageUrlQuery(
#         url=image_url,
#         k_nearest_neighbors=top,
#         fields="image_vector",
#         weight=0.5  # Adjust weight as needed
#     )

#     results = search_client.search(
#         search_text=query,  
#         vector_queries=[text_vector_query, image_vector_query],
#         select=["content"],
#         top=top
#     )
#     return [doc["content"] for doc in results]


# Step 3: Generate a multimodal RAG-based answer using retrieved text and an image input
def generate_multimodal_rag_response(query: str, image_url: str):
    # Retrieve text context from search
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)

    # Build a prompt that combines the retrieved context with the user query
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    # Create a chat request that includes both text and image input
    response = chat_client.complete(
        messages=[
            SystemMessage(content="You are a helpful assistant that can process both text and images."),
            UserMessage(
                content=[
                    TextContentItem(text=prompt),
                    ImageContentItem(image_url=ImageUrl(url=image_url, detail=ImageDetailLevel.HIGH)),
                ]
            ),
        ]
    )
    return response.choices[0].message.content

# Example usage
user_query = "Can you search for similar items to this shoe?"
sample_image_url = "https://images.unsplash.com/photo-1542291026-7eec264c27ff?q=80&w=1770&auto=format&fit=crop&ixlib=rb-4.0.3"
answer = generate_multimodal_rag_response(user_query, sample_image_url)
print(f"Q: {user_query}\nA: {answer}")

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
അറിയിപ്പ്:
ഈ പ്രമാണം AI പരിഭാഷാ സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് പരിഭാഷപ്പെടുത്തിയതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചാലും, ഓട്ടോമേറ്റഡ് (സ്വയംക്രമിത) പരിഭാഷകളിൽ പിശകുകളോ അസാധുതകളോ ഉണ്ടായിരിക്കാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. സ്വഭാഷയിലെ ഓറിജിനൽ പ്രമാണത്തെ അധികാരപരമായ ഉറവിടമായി കരുതണം. നിർണ്ണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ പരിഭാഷ ശുപാര്‍ശ ചെയ്യപ്പെടുന്നു. ഈ പരിഭാഷ ഉപയോഗിച്ചതിന്റെ ഫലമായി ഉണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണങ്ങൾക്കും വ്യാഖ്യാനക്കേടുകൾക്കും ഞങ്ങൾ ഉത്തരവാദിത്തം സ്വീകരിക്കുന്നില്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
